# About the Project:

In this project, we will classify a Heart Disease Dataset to predict people who have the disease.
The models used for this task are MLP and CNN that applied with Keras; also, we used a metaheuristic algorithm(HHO) to optimize the number of nodes in each layer.

## Introduction

WHO announced that cardiovascular diseases is the top one killer over the world. There are seventeen million people died from it every year, especially heart disease. Prevention is better than cure. If we can evaluate the risk of every patient who probably has heart disease, that is, not only patients but also everyone can do something earlier to keep illness away.

This dataset is a real data including important features of patients. This time we will build the predictable model by XGBoost library. Before predict the test dataset, use concept of crossvalid to find the optimised parameters. after that, the model can calculate the weight of each features, so we can easily understand which feature is more influent than others.

Confusion matrix is a common technique to figure out the accuracy of the model. From the standpoint of medicine, the recall rate is more important than precision rate because no one want to be misdiagnosed if the one actually have heart disease. So we will check the recall performance. After that, roc curve can help us evaluate the model, and then we'll explore the features if the model is good enough.

"Shap" is a powerful library to know whether each feature is positive or negative relationship with heart disease. At the end of the report, we double confirm the result, and we can also know which feature affect each patient the most.

## Exploratory Analysis

### There are thirteen features and one target as below:

* age: The person's age in years

* sex: The person's sex (1 = male, 0 = female)

* cp: The chest pain experienced (Value 1: typical angina, Value 2: atypical angina, Value 3: non-anginal pain, Value 4: asymptomatic)

* trestbps: The person's resting blood pressure (mm Hg on admission to the hospital)

* chol: The person's cholesterol measurement in mg/dl

* fbs: The person's fasting blood sugar (> 120 mg/dl, 1 = true; 0 = false)

* restecg: Resting electrocardiographic measurement (0 = normal, 1 = having ST-T wave abnormality, 2 = showing probable or definite left ventricular hypertrophy by Estes' criteria)

* thalach: The person's maximum heart rate achieved

* exang: Exercise induced angina (1 = yes; 0 = no)

* oldpeak: ST depression induced by exercise relative to rest

* slope: the slope of the peak exercise ST segment (Value 1: upsloping, Value 2: flat, Value 3: downsloping)

* ca: The number of major vessels (0-3)

* thal: A blood disorder called thalassemia (3 = normal; 6 = fixed defect; 7 = reversable defect)

* target: Heart disease (0 = no, 1 = yes)

# Importing Libraries and Data

In [ ]:
pip install pygad

In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import ExtraTreesClassifier
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
import pygad.kerasga

In [ ]:
df = pd.read_csv('./heart.csv')
df

In [ ]:
X = df.drop("target", axis=1)
y = df["target"]

# Information About Data

In [ ]:
def explore_dataframe(df):
    print("-"*30)
    print("Overall Info of Columns :")
    print(df.info()) 
    print("-"*30)
    print("Quantity of Nans :")
    print(df.isna().sum()) 
    print("-"*30)
    print("Percentage of Nans :")
    print(df.isna().mean()*100) 
    print("-"*30)
    print("Variable Types :")
    print(df.dtypes)
    print("-"*30)

In [ ]:
explore_dataframe(df)

# Visualization 

In [ ]:
df_target = df.groupby("target").size()
plt.pie(df_target.values, labels = ["No Heart Disease", "With Heart Disease"], autopct='%1.1f%%', radius = 1.5, textprops = {"fontsize" : 16}) 
plt.show()

In [ ]:
df_sex = df.groupby(["sex","target"]).size()
plt.pie(df_sex.values, labels = ["Female,No Heart Disease", "Female,With Heart Disease", "Male,No Heart Disease", "Male,With Heart Disease"],autopct='%1.1f%%',radius = 1.5, textprops = {"fontsize" : 16})
plt.show()

In [ ]:
plt.hist([df[df.target==0].age, df[df.target==1].age], bins = 20, alpha = 0.5, label = ["No Heart Disease","With Heart Disease"])
plt.xlabel("age")
plt.ylabel("percentage")
plt.legend()
plt.show()

In [ ]:
plt.hist([df[df.target==0].chol, df[df.target==1].chol], bins = 20, alpha = 0.5, label = ["No Heart Disease","With Heart Disease"])
plt.xlabel("chol")
plt.ylabel("percentage")
plt.legend()
plt.show()

In [ ]:
plt.hist([df[df.target==0].trestbps, df[df.target==1].trestbps], bins = 20, alpha = 0.5, label = ["No Heart Disease","With Heart Disease"])
plt.xlabel("trestbps")
plt.ylabel("percentage")
plt.legend()
plt.show()

In [ ]:
plt.hist([df[df.target==0].thalach, df[df.target==1].thalach], bins = 20, alpha = 0.5, label = ["No Heart Disease","With Heart Disease"])
plt.xlabel("thalach")
plt.ylabel("percentage")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.title('Heatmap Displaying the Relationship Betweennthe Features of the Data',fontsize=13)
sns.heatmap(df.corr(), annot=True, cmap='Blues')
hm = sns.heatmap(df.corr(), annot=True, cmap='Blues')

## Feature Importance 

You can get the feature importance of each feature of your dataset by using the feature importance property of the model.
Feature importance gives you a score for each feature of your data, the higher the score more important or relevant is the feature towards your output variable.
Feature importance is an inbuilt class that comes with Tree Based Classifiers, we will be using Extra Tree Classifier for extracting the top 10 features for the dataset.

In [ ]:
model = ExtraTreesClassifier()
model.fit(X,y)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_ , index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

In [ ]:
X = df[['ca','cp','thal','exang','thalach','oldpeak','age','age','slope','trestbps','chol']]

In [ ]:
X = df.loc[:, df.columns != 'target']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scl=MinMaxScaler()
X=scl.fit_transform(X)

In [ ]:
X = tf.keras.utils.normalize(X, axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.30, random_state=0, shuffle=True)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

### **HHO**


In [ ]:
import random
import numpy
import math


def Levy(dim):
    beta=1.5
    sigma=(math.gamma(1+beta)*math.sin(math.pi*beta/2)/(math.gamma((1+beta)/2)*beta*2**((beta-1)/2)))**(1/beta) 
    u= 0.01*numpy.random.randn(dim)*sigma
    v = numpy.random.randn(dim)
    zz = numpy.power(numpy.absolute(v),(1/beta))
    step = numpy.divide(u,zz)
    return step



def HHO(objf,lb,ub,dim,SearchAgents_no,Max_iter):

    #dim=30
    #SearchAgents_no=50
    #lb=-100
    #ub=100
    #Max_iter=500
        
    
    # initialize the location and Energy of the rabbit
    Rabbit_Location=numpy.zeros(dim)
    Rabbit_Energy=float("inf")  #change this to -inf for maximization problems
    
    if not isinstance(lb, list):
        lb = [lb for _ in range(dim)]
        ub = [ub for _ in range(dim)]
    lb = numpy.asarray(lb)
    ub = numpy.asarray(ub)
         
    #Initialize the locations of Harris' hawks
    X=numpy.zeros((SearchAgents_no, dim))
    for i in range(SearchAgents_no):
        for d in range(dim):
            if(numpy.random.rand()>0.5):
                X[i,d] = 1;
            
    fitR = math.inf; 
    fit  = numpy.zeros((N)); 
    Y    = numpy.zeros((dim)); 
    Z    = numpy.zeros((dim));
    
    curve =numpy.zeros(max_Iter);  
    t = 1; 
    
    
    #---Iteration start-------------------------------------------------
    while t <= max_Iter:
      for i in range(SearchAgents_no):
        fit[i] =objf(X[i,:]) #fun(feat,label,X(i,:),HO);
        if fit[i] < fitR:
          fitR = fit[i]
          Xrb  = X[i,:]
    
      X_mu = numpy.mean(X,0);
      for i in range(SearchAgents_no):
        E0 = -1 + 2 * numpy.random.rand();
        E  = 2 * E0 * (1 - (t / max_Iter)); 
        if abs(E) >= 1:
          q = numpy.random.rand(); 
          if q >= 0.5:
            k  = numpy.random.randint(1,SearchAgents_no);
            r1 = numpy.random.rand();
            r2 = numpy.random.rand();
            for d in range(dim):
              Xn = X[k,d] - r1 * abs(X[k,d] - 2 * r2 * X[i,d]);
              S  = 1 / (1 + numpy.exp(-Xn));
              if numpy.random.rand() < S:
                X[i,d]= 1
              else:
                X[i,d] = 0
 
          elif q < 0.5:
            r3 = numpy.random.rand();
            r4 = numpy.random.rand();
            for d in range(dim):
              Xn = (Xrb[d]- X_mu[d]) - r3 * (lb + r4 * (ub - lb));
              S  = 1 / (1 + numpy.exp(-Xn));
              if numpy.random.rand() < S[0]:
                X[i,d] = 1;
              else:
                X[i,d] = 0;
        elif abs(E) < 1:
          J = 2 * (1 - numpy.random.rand());
          r = numpy.random.rand();
          if r >= 0.5  and  abs(E) >= 0.5:
            for  d in range(dim):
              DX = Xrb[d] - X[i,d];
              Xn = DX - E * abs(J * Xrb[d]- X[i,d]);
              S  = 1 / (1 + numpy.exp(-Xn));
              if numpy.random.rand() < S:
                 X[i,d]= 1
              else:
                X[i,d] = 0;
 
          elif r >= 0.5  and  abs(E) < 0.5:
            for  d in range(dim):
              DX = Xrb[d] - X[i,d];
              Xn = Xrb[d]- E * abs(DX);
              S  = 1 / (1 + numpy.exp(-Xn));
              if numpy.random.rand() < S:
                 X[i,d]= 1
              else:
                 X[i,d]= 0
 
          elif r < 0.5  and  abs(E) >= 0.5:
            LF = Levy(dim); 
            for  d in range(dim):
              Yn = Xrb[d] - E * abs(J * Xrb[d] - X[i,d]);
              S  = 1 / (1 + numpy.exp(-Yn));
              if numpy.random.rand() < S:
                Y[d] = 1;
              else:
                Y[d] = 0;
 
              Zn = Y[d] + numpy.random.rand() * LF[d];
              S  = 1 / (1 + numpy.exp(-Zn));
              if numpy.random.rand() < S:
                Z[d]= 1;
              else:
                Z[d] = 0;
 
            fitY =objf(Y)# fun(feat,label,Y,HO);
            fitZ =objf(Z)# fun(feat,label,Z,HO);
            if fitY <= fit[i]:
              fit[i] = fitY; 
              X[i,:]= Y;
 
            if fitZ <= fit[i]:
              fit[i] = fitZ;
              X[i,:]= Z;
 
          elif r < 0.5  and  abs(E) < 0.5:
            LF = Levy(dim); 
            for  d in range(dim):
              Yn = Xrb[d] - E * abs(J * Xrb[d] - X_mu[d]);
              S  = 1 / (1 + numpy.exp(-Yn));
              if numpy.random.rand() < S:
                Y[d]= 1;
              else:
                 Y[d] = 0;
 
              Zn = Y[d] + numpy.random.rand() * LF[d];
              S  = 1 / (1 + numpy.exp(-Zn));
              if numpy.random.rand() < S:
                Z[d] = 1;
              else:
                Z[d] = 0;
 
            fitY =objf(Y)# fun(feat,label,Y,HO); 
            fitZ =objf(Z)# fun(feat,label,Z,HO);
            if fitY <= fit[i]:
              fit[i] = fitY; 
              X[i,:] = Y;
        
            if fitZ <= fit[i]:
              fit[i] = fitZ; 
              X[i,:] = Z;
  
    
  
      curve[t-1] = fitR; 
      print('\nIteration{0} Best (BHHO)= {1}'.format(t,curve[t-1]))
      t = t + 1;

        
      


    
    return curve,Xrb

# **MLP Model**

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

def Create_mlp_model(data,array_ner):
  model = keras.Sequential([
      layers.Dense(array_ner[0], activation='relu', input_shape=[data.shape[1]]),
      layers.Dropout(0.5),
      layers.BatchNormalization(),
      layers.Dense(array_ner[1], activation='relu'), 
      layers.Dropout(0.5),
      layers.BatchNormalization(),
      layers.Dense(array_ner[2], activation='relu'), 
      #layers.Dropout(0.3),
      layers.BatchNormalization(),
      layers.Dense(1, activation='sigmoid'),
  ])
  model.compile(
      optimizer='adam',
      loss='binary_crossentropy',
      metrics=['binary_accuracy'])
  return model
  


In [ ]:
def funcost(x):
  x[x>=0.5]=1;
  x[x<0.5]=0;
  x=x.astype(int)
  print(x)
  array_ner=np.zeros(3,dtype=int)
  array_ner[0]=int(''.join([str(elem) for elem in x[0:10]]),2)
  array_ner[1]=int(''.join([str(elem) for elem in x[10:20]]),2)
  array_ner[2]= int(''.join([str(elem) for elem in x[20:30]]),2)
  print(array_ner)
  
  model=Create_mlp_model(data_test,array_ner)
  model.fit(data_train,lbl_train,epochs=100,batch_size=20, verbose=0)
  loss, accuracy = model.evaluate(data_test, lbl_test, verbose=0)

  acc=accuracy
  print(acc)
  return 1/acc

In [ ]:
data_train, data_test, lbl_train, lbl_test = train_test_split( X, y, test_size=0.50, random_state=10, shuffle=True)

In [ ]:
N= 10; 
max_Iter = 100;
objf=funcost
lb=0
ub=1
dim = 30;
curve,best_sol=HHO(objf,lb,ub,dim,N,max_Iter)


In [ ]:
x=best_sol
x=x.astype(int)
array_ner=np.zeros(3,dtype=int)
array_ner[0]=int(''.join([str(elem) for elem in x[0:10]]),2)
array_ner[1]=int(''.join([str(elem) for elem in x[10:20]]),2)
array_ner[2]= int(''.join([str(elem) for elem in x[20:30]]),2)
print(array_ner)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(array_ner[0], activation='relu', input_shape=[X_train.shape[1]]),
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(array_ner[1], activation='relu'), 
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(array_ner[2], activation='relu'), 
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(1, activation='sigmoid'),
])
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=10,
    epochs=100,
    verbose=1, # hide the output because we have so many epochs
)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df

In [ ]:
history_df = pd.DataFrame(history.history)
# Start the plot at epoch 5
history_df.loc[:, ['loss', 'val_loss']].plot()
history_df.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot()

print(("Best Validation Loss: {:0.4f}" +\
      "\nBest Validation Accuracy: {:0.4f}")\
      .format(history_df['val_loss'].min(), 
              history_df['val_binary_accuracy'].max()))

**CNN Model**

In [ ]:
def Create_CCN_model(array_ner):
  
  N,dim1=X_train.shape[0],X_train.shape[1]
  input_layer=keras.layers.Input(shape=(dim1,1),name="input_layer")
  # array_ner[0]=100
  # array_ner[1]=100
  # array_ner[2]=100
  array_ner=[270,810,760,300,0]
  model=keras.models.Sequential([input_layer,
                                keras.layers.Conv1D(filters=array_ner[0],kernel_size=(2),strides=(1),padding="same",name="conv1"),
                                keras.layers.MaxPool1D(pool_size=(2),name="maxpool1"),
                                keras.layers.Dropout(0.2),
                                keras.layers.BatchNormalization(),
                                keras.layers.Conv1D(filters=array_ner[1],kernel_size=(2),strides=(1),padding="same",name="conv2"),
                                keras.layers.MaxPool1D(pool_size=(2),name="maxpool2"),
                                keras.layers.Dropout(0.2),
                                keras.layers.BatchNormalization(),
                                keras.layers.Conv1D(filters=array_ner[2],kernel_size=(2),strides=(1),padding="same",name="conv3"),
                                keras.layers.MaxPool1D(pool_size=(2),name="maxpool3"),
                                keras.layers.Dropout(0.2),
                                keras.layers.BatchNormalization(),
                                keras.layers.Flatten(),
                                keras.layers.Dense(array_ner[3],activation="relu"),
                                keras.layers.Dense(1, activation='sigmoid',name="out_layer")
                                
                                ]
                                
                              )
  
  model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],)
  return model

In [ ]:
data_train_re=data_train.reshape(data_train.shape[0],data_train.shape[1],1)
data_test_re=data_test.reshape(data_test.shape[0],data_test.shape[1],1)

In [ ]:
def funcost(x):
  x[x>=0.5]=1;
  x[x<0.5]=0;
  x=x.astype(int)
  print(x)
  array_ner=np.zeros(3,dtype=int)
  array_ner[0]=int(''.join([str(elem) for elem in x[0:7]]),2)
  array_ner[1]=int(''.join([str(elem) for elem in x[7:14]]),2)
  array_ner[2]= int(''.join([str(elem) for elem in x[14:21]]),2)
  print(array_ner)
  
  model=Create_CCN_model(array_ner)
  model.fit(data_train_re,lbl_train,epochs=100,batch_size=20, verbose=0)
  loss, accuracy = model.evaluate(data_test_re, lbl_test, verbose=0)

  acc=accuracy
  print(acc)
  return 1/acc

In [ ]:
N= 2; 
max_Iter = 2;
objf=funcost
lb=0
ub=1
dim = 21;
curve,best_sol=HHO(objf,lb,ub,dim,N,max_Iter)


In [ ]:
x=best_sol
x=x.astype(int)

array_ner=np.zeros(3,dtype=int)
array_ner[0]=int(''.join([str(elem) for elem in x[0:10]]),2)
array_ner[1]=int(''.join([str(elem) for elem in x[10:20]]),2)
array_ner[2]= int(''.join([str(elem) for elem in x[20:30]]),2)
print(array_ner)

In [ ]:
X_train_re=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test_re=X_test.reshape(X_test.shape[0],X_test.shape[1],1)

In [ ]:
model=Create_CCN_model(array_ner)

In [ ]:
history = model.fit(
    X_train_re, y_train,
    validation_data=(X_test_re, y_test),
    batch_size=10,
    epochs=100,
    verbose=1, # hide the output because we have so many epochs
)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df

In [ ]:
history_df = pd.DataFrame(history.history)
# Start the plot at epoch 5
history_df.loc[:, ['loss', 'val_loss']].plot()
history_df.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot()

print(("Best Validation Loss: {:0.4f}" +\
      "\nBest Validation Accuracy: {:0.4f}")\
      .format(history_df['val_loss'].min(), 
              history_df['val_binary_accuracy'].max()))